## Normalisation factor - redshift dependent

In [1]:
from scipy.optimize import curve_fit, minimize
from scipy.interpolate import RegularGridInterpolator
from scipy import integrate
import emcee
import corner
import time
from multiprocessing import cpu_count, Pool
import os
import h5py 
import numpy as np
import astropy.table as aTable
from astropy.cosmology import Planck13
from astropy.cosmology import FlatLambdaCDM

import matplotlib as mpl
import matplotlib.pyplot as plt

# mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

In [2]:
def mass_completeness_limit(z):
    f, b, c = [-1.34199453, 13.90578909,  8.53522654]
    return 4*np.pi*f*z**2 + b*z + c


def zmax_mass_completeness_limit(m):
    f, b, c = [-1.34199453, 13.90578909, 8.53522654]

    discriminant = -16*np.pi*f*c + 16*np.pi*f*m + b**2
    sqrt_discriminant = np.sqrt(discriminant)

    z1 = (-b + sqrt_discriminant) / (8 * np.pi * f)
    
    return z1


def region_limits(z_i, m_i):
    if m_i > 11.401850153336317:
        threshold = 0.4
        return np.array([
        [0.01, mass_completeness_limit(z_i)],
        [z_i, mass_completeness_limit(z_i)],
        [threshold, mass_completeness_limit(threshold)],
        [threshold, 13.],
        [0.01, 13.]])
    return np.array([
        [0.01, mass_completeness_limit(z_i)],
        [z_i, mass_completeness_limit(z_i)],
        [zmax_mass_completeness_limit(m_i), m_i],
        [zmax_mass_completeness_limit(m_i), 13.],
        [0.01, 13]])



def integral_limits(z_i, m_i):
    
    zmax_thr = 0.4
    mmax_thr = 13.
    m_lim_i = mass_completeness_limit(z_i)
    zmax_thr_i = zmax_mass_completeness_limit(mass_completeness_limit(zmax_thr))
    
    if m_i > 11.401850153336317:
        return np.array([
            [0.01, z_i],
            [m_lim_i, mmax_thr],
            [z_i, zmax_thr_i],
            [mass_completeness_limit ,mmax_thr]])
    
    z_max_i = zmax_mass_completeness_limit(m_i)
    
    return np.array([
        [0.01, z_i],
        [m_lim_i, mmax_thr],
        [z_i, z_max_i],
        [mass_completeness_limit, mmax_thr]])

    

    
def smf_single_schechter_sty(x, z, a0, a1, a2, a3):
    logM = a0 + a1*z
    alpha1 = a2 + a3*z
    
    term0 = np.exp(-10 ** (x-logM[:,None]))
    term1 = 10 ** ((alpha1+1)[:,None]*(x - logM[:,None]))
    return term0 * term1



def smf_single_schechter_integral(x, z, a0, a1, a2, a3):
    logM = a0 + a1*z
    alpha1 = a2 + a3*z
    term0 = np.exp(-10 ** (x-logM))
    term1 = 10 ** ((alpha1+1)*(x - logM))
    return term0 * term1



def log_likelihood(a0, a1, a2, a3, I, w, z, x):
    q = smf_single_schechter_sty(x, z, a0, a1, a2, a3)
    a = np.log10(np.sum(q, axis=1)) - np.log10(np.array(I))
    return a * w




def log_prior(theta):
    a0, a1, a2, a3 = theta
    if 9.5 < a0 < 13.5 and \
        -4. < a1 < 4. and \
       -2.5 < a2 < -0.5  and \
       1.5 < a3 < 6.:
        return 0
    return -np.inf



# a1, a3 free parameters
# def log_prior(theta):
#     a0, a1, a2, a3 = theta
#     if 9.5 < a0 < 13.5 and \
#        -2.5 < a2 < -0.5:
#         return 0
#     return -np.inf



def posterior(theta, integral_limits_list, w, z, x):
    a0, a1, a2, a3 = theta
    I = []
    for limit in integral_limits_list:
        I.append(integral_calculation(limit, a0, a1, a2, a3))
        
    l = log_likelihood(a0, a1, a2, a3, I, w, z, x)
    return log_prior(theta) + np.sum(l)




def integral_calculation(limits, a0, a1, a2, a3):
    result = 0
    result += integrate.dblquad(smf_single_schechter_integral, limits[0][0],
                                limits[0][1], limits[1][0], limits[1][1], args=(a0, a1, a2, a3), 
                                epsabs=1e-3, epsrel=1e-3)[0]
    result += integrate.dblquad(smf_single_schechter_integral, limits[2][0],
                            limits[2][1], limits[3][0] , limits[3][1], args=(a0, a1, a2, a3), 
                                epsabs=1e-3, epsrel=1e-3)[0]
    
    return result




def integral_calculation2(a0, a1, a2, a3, m, z):
    limits = integral_limits(z, m)
    result = 0
    result += integrate.dblquad(smf_single_schechter_integral, limits[0][0],
                                limits[0][1], limits[1][0], limits[1][1], args=(a0, a1, a2, a3), 
                                epsabs=1e-3, epsrel=1e-3)[0]
    result += integrate.dblquad(smf_single_schechter_integral, limits[2][0],
                            limits[2][1], limits[3][0] , limits[3][1], args=(a0, a1, a2, a3), 
                                epsabs=1e-3, epsrel=1e-3)[0]
    
    return result




def smf_single_schechter_integral_norm(x, z, z0, a0, a1, a2, a3):
    logM = a0 + a1*z0
    alpha1 = a2 + a3*z0
    term0 = np.exp(-10 ** (x-logM))
    term1 = 10 ** ((alpha1+1)*(x - logM))
    return term0 * term1




def integral_calculation_m(m, z, z0, a0, a1, a2, a3):
    return integrate.quad(smf_single_schechter_integral_norm, mass_completeness_limit(z), 13., args=(z, z0, a0, a1, a2, a3))[0]

                           
                           
                           

def v_tot_z(f_area, z):
    v_min = Planck13.comoving_volume(0.01).value * Planck13.h**3 * f_area
    v_max = Planck13.comoving_volume(z).value * Planck13.h**3 * f_area
    return v_max  - v_min



def normalisation(f_area, x, x_median, w_spec, z, z0, best_params):
    a0, a1, a2, a3 = best_params
    
    v_tot = v_tot_z(f_area,z0)
    
    mask_z = (z < z0)
    z_new = z[mask_z]
    x_new = x[mask_z]
    x_median_new = x_median[mask_z]
    w_spec_new = w_spec[mask_z]
    
    weights = []
    
    for i in range(x_median_new.shape[0]):
        weights.append(w_spec_new[i]/integral_calculation_m(x_median_new[i], z_new[i], z0, a0, a1, a2, a3))
        
    return (1/v_tot) *np.sum(weights)



# def normalisation(f_area, x, x_median, w_spec, z, z0, best_params):
#     a0, a1, a2, a3 = best_params
    
#     v_tot = v_tot_z(f_area,z0)
    
#     mask_z = (z < z0)
#     z_new = z[mask_z]
#     x_new = x[mask_z]
#     x_median_new = x_median[mask_z]
#     w_spec_new = w_spec[mask_z]
#     vmax_new = vmax[mask_z]
    
#     weights = []
    
#     for i in range(x_median_new.shape[0]):
#         weights.append(w_spec_new[i]/integral_calculation2(a0, a1, a2, a3, x_median_new[i], z_new[i]))
        
#     return (1/v_tot) *np.sum(weights)
        
    
    
    

# def posterior(theta, Mlim, w, z, x):
#     a0, a1, a2, a3 = theta
#     I = []
#     for i in range(Mlim.shape[0]):
#         I.append(integrate.dblquad(smf_single_schechter_integral, 0.01, z[i], Mlim[i], 15., args=(a0, a1, a2, a3))[0])
        
#     l = log_likelihood(a0, a1, a2, a3, I, w, z, x)
#     return log_prior(theta) + np.sum(l)

In [3]:
cosmo = FlatLambdaCDM(H0=67.8, Om0=0.307)

# Read hdf5 for BGS data
bgs = aTable.Table.read('BGS_ANY_full.provabgs.lite.hdf5')
is_bgs_bright = bgs['is_bgs_bright']
is_bgs_faint = bgs['is_bgs_faint']

bgs = bgs[bgs['is_bgs_bright']]

# Gathering data
# Exluding galaxies for z < 0.01 and z > 0.4 because I don't have the mass completeness limit for those
mask_zlim = (bgs['Z_HP'].data > 0.01) & (bgs['Z_HP'].data < 0.4)

z_tot = bgs['Z_HP'].data[mask_zlim]
x_tot = bgs['provabgs_logMstar'].data[mask_zlim]
x_median_tot = np.median(x_tot, axis=1)
w_zfail_tot = bgs['provabgs_w_zfail'].data[mask_zlim]
w_fib_tot = bgs['provabgs_w_fibassign'].data[mask_zlim]
vmax_tot = bgs['Vmax'].data[mask_zlim]


mass_comp_lim = mass_completeness_limit(z_tot)
mask_mlim = []
for i in range(len(x_median_tot)):
    mask_mlim.append(x_median_tot[i] > mass_comp_lim[i])
    

mask = (w_zfail_tot > 0) & (mask_mlim)

z = z_tot[mask].astype(np.float32)
x = x_tot[mask].astype(np.float32)
x_median = x_median_tot[mask].astype(np.float32)
w_zfail = w_zfail_tot[mask].astype(np.float32)
w_fib = w_fib_tot[mask].astype(np.float32)
vmax = vmax_tot[mask].astype(np.float32)


f_area = (173.641/(4.*np.pi*(180/np.pi)**2))
v_zmin = Planck13.comoving_volume(0.01).value * Planck13.h**3 * f_area # (Mpc/h)^3
v_zmax = Planck13.comoving_volume(0.09).value * Planck13.h**3 * f_area # (Mpc/h)^3
v_sub = v_zmax - v_zmin


# w_spec * 1/Vmax
w = (w_zfail*w_fib) * v_sub / (vmax.clip(v_zmin, v_zmax) - v_zmin)
n = np.sum(w)/v_sub

# Spectroscopic weights
w_spec = (w_zfail*w_fib)

In [4]:
z0 = 0.035
# best_params = [10., 3.2, -1.5, 1.5]
best_params = [10.13157873,  5.07902063, -1.76807801,  0.02749427]
# best_params = [10.29956427,  3.69867378, -1.68922073,  0.02706255]

normalisation(f_area, x, x_median, w_spec, z, z0, best_params)

0.021697444966845558

In [5]:
integrate.quad(smf_single_schechter_integral, mass_completeness_limit(z0), 13., args=(z0, 10., 3.2, -1.5, 1.5))

(1.6752611706063938, 5.593748252791242e-09)

-----
-----
-----

In [ ]:
mask_z = (z < z0)

z_new = z[mask_z]
x_new = x[mask_z]
x_median_new = x_median[mask_z]
w_spec_new = w_spec[mask_z]
vmax_new = vmax[mask_z]

In [ ]:
weights = []
a0, a1, a2, a3 = best_params
for i in range(x_median_new.shape[0]):
    weights.append((vmax_new[i] - v_zmin)*w_spec_new[i]/integral_calculation2(a0, a1, a2, a3, x_median_new[i], z_new[i]))

In [ ]:
np.sum(weights)/v_tot_z(f_area, z0)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(z_tot, x_median_tot, 'o', markersize=0.5)
# plt.plot(z_new, x_median_new, 'o', markersize=0.1)
plt.xlim(0, 0.6)

In [ ]:
z0 = np.linspace(0.02, 0.6, 4)

best_params = [10., 3.2, -1.5, 1.5]

plt.figure(figsize=(6,6))
for z0_i in z0:
    plt.plot(z0_i, normalisation(f_area, x, x_median, w_spec, z, z0_i, best_params), 'o')


In [ ]:
# z0 = 0.2
mask_z = (z < z0)

In [ ]:
a0, a1, a2, a3 = [10., 3.2, -1.5, 1.5]

z0 = 0.2
v_tot = v_tot_z(f_area,z0)

mask_z = (z < z0)
z_new = z[mask_z]
x_new = x[mask_z]
x_median_new = x_median[mask_z]
w_spec_new = w_spec[mask_z]

weights = []

for i in range(x_median_new.shape[0]):
    weights.append(w_spec_new[i]/integral_calculation2(a0, a1, a2, a3, x_median_new[i], z_new[i]))

(1/v_tot) *np.sum(weights)

In [ ]:
integral_calculation2(a0, a1, a2, a3, x_median_new[2], z[2])

In [ ]:
weights

In [ ]:
z0 = 0.2
best_params = [10., 3.2, -1.5, 1.5]
normalisation(f_area, x, x_median, w_spec, z, z0, best_params)

In [ ]:
nn = 10

a0_lin = np.linspace(9.5, 13.5, nn)
a1_lin = np.linspace(2.5, 6., nn)
a2_lin = np.linspace(-2.5, -0.5, nn)
a3_lin = np.linspace(0, 3., nn)
m_lin = np.linspace(x_median.min(), x_median.max(), nn)
z_lin = np.linspace(z.min(), z.max(), nn)

In [ ]:
# integral_calculation2(a0_lin[0], a1_lin[0], a2_lin[0], a3_lin[0], z_lin[0], m_lin[0])

In [ ]:
# Create a 10x10x10 cube meshgrid
nn = 6

a0_lin = np.linspace(9.5, 13.5, nn)
a1_lin = np.linspace(2.5, 6., nn)
a2_lin = np.linspace(-2.5, -0.5, nn)
a3_lin = np.linspace(0, 3., nn)
m_lin = np.linspace(x_median.min(), x_median.max(), nn)
z_lin = np.linspace(z.min(), z.max(), nn)

grid_a0, grid_a1, grid_a2, grid_a3, grid_m, grid_z = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, m_lin, z_lin, indexing='ij')

# Define the subgrid size
n_division = 2
subgrid_size = 3

# Divide the cube meshgrid into eight adjacent subgrids
subgrids = []

for o in range(n_division):
    for s in range(n_division):
        for q in range(n_division):
            for k in range(n_division):
                for j in range(n_division):
                    for i in range(n_division):
                        subgrid_a0 = grid_a0[i*subgrid_size:(i+1)*subgrid_size, j*subgrid_size:(j+1)*subgrid_size, k*subgrid_size:(k+1)*subgrid_size, q*subgrid_size:(q+1)*subgrid_size, s*subgrid_size:(s+1)*subgrid_size, o*subgrid_size:(o+1)*subgrid_size]
                        subgrid_a1 = grid_a1[i*subgrid_size:(i+1)*subgrid_size, j*subgrid_size:(j+1)*subgrid_size, k*subgrid_size:(k+1)*subgrid_size, q*subgrid_size:(q+1)*subgrid_size, s*subgrid_size:(s+1)*subgrid_size, o*subgrid_size:(o+1)*subgrid_size]
                        subgrid_a2 = grid_a2[i*subgrid_size:(i+1)*subgrid_size, j*subgrid_size:(j+1)*subgrid_size, k*subgrid_size:(k+1)*subgrid_size, q*subgrid_size:(q+1)*subgrid_size, s*subgrid_size:(s+1)*subgrid_size, o*subgrid_size:(o+1)*subgrid_size]
                        subgrid_a3 = grid_a3[i*subgrid_size:(i+1)*subgrid_size, j*subgrid_size:(j+1)*subgrid_size, k*subgrid_size:(k+1)*subgrid_size, q*subgrid_size:(q+1)*subgrid_size, s*subgrid_size:(s+1)*subgrid_size, o*subgrid_size:(o+1)*subgrid_size]
                        subgrid_m = grid_m[i*subgrid_size:(i+1)*subgrid_size, j*subgrid_size:(j+1)*subgrid_size, k*subgrid_size:(k+1)*subgrid_size, q*subgrid_size:(q+1)*subgrid_size, s*subgrid_size:(s+1)*subgrid_size, o*subgrid_size:(o+1)*subgrid_size]
                        subgrid_z = grid_z[i*subgrid_size:(i+1)*subgrid_size, j*subgrid_size:(j+1)*subgrid_size, k*subgrid_size:(k+1)*subgrid_size, q*subgrid_size:(q+1)*subgrid_size, s*subgrid_size:(s+1)*subgrid_size, o*subgrid_size:(o+1)*subgrid_size]
                        subgrids.append((subgrid_a0, subgrid_a1, subgrid_a2, subgrid_a3, subgrid_m, subgrid_z))

In [ ]:
results = np.zeros((grid_a0.shape[0], grid_a1.shape[1], grid_a2.shape[2], grid_a3.shape[3], grid_m.shape[4], grid_z.shape[5]))

for o in range(len(z_lin)):
    for s in range(len(m_lin)):
        for q in range(len(a3_lin)):
            for k in range(len(a2_lin)):
                for j in range(len(a1_lin)):
                    for i in range(len(a0_lin)):
                        results[i,j,k,q,s,o] = integral_calculation2(grid_a0[i,j,k,q,s,o], grid_a1[i,j,k,q,s,o], grid_a2[i,j,k,q,s,o],
                                                                     grid_a3[i,j,k,q,s,o], grid_m[i,j,k,q,s,o], grid_z[i,j,k,q,s,o])

In [ ]:
results_subgrids = []

for subgrid in subgrids:
    results_subgrid = np.zeros((subgrid[0][0].shape[0], subgrid[0][0].shape[0], subgrid[0][0].shape[0], subgrid[0][0].shape[0], subgrid[0][0].shape[0], subgrid[0][0].shape[0]))
    
    for o in range(subgrid_size):
        for s in range(subgrid_size):
            for q in range(subgrid_size):
                for k in range(subgrid_size):
                    for j in range(subgrid_size):
                        for i in range(subgrid_size):
                            results_subgrid[i,j,k,q,s,o] = integral_calculation2(subgrid[0][i,j,k,q,s,o], subgrid[1][i,j,k,q,s,o],
                                                                                 subgrid[2][i,j,k,q,s,o], subgrid[3][i,j,k,q,s,o],
                                                                                 subgrid[4][i,j,k,q,s,o], subgrid[5][i,j,k,q,s,o])

    results_subgrids.append(results_subgrid)

In [ ]:
def concatenate_subgrids(subgrids, n_division):
    n_subgrids = len(subgrids)
    strips_array = []
    bases_array = []
    bases2_array = []
    bases3_array = []
    bases4_array = []
    
    for i in range(0, n_subgrids, n_division):
        strips_array.append(np.concatenate((subgrids[i:i + n_division]), axis=0))
    
        
    for j in range(0, len(strips_array)-1, n_division):
        bases_array.append(np.concatenate((strips_array[j:j + n_division]), axis=1))
        
        
    for k in range(0, len(bases_array)-1, n_division):
        bases2_array.append(np.concatenate((bases_array[k:k + n_division]), axis=2))
        
        
    for a in range(0, len(bases2_array)-1, n_division):
        bases3_array.append(np.concatenate((bases2_array[a:a + n_division]), axis=3))
    
    
    for b in range(0, len(bases3_array)-1, n_division):
        bases4_array.append(np.concatenate((bases3_array[b:b + n_division]), axis=4))
        
        
    return np.concatenate(bases4_array, axis=5)

In [ ]:
results.shape == concatenate_subgrids(results_subgrids, n_division).shape

In [ ]:
results == concatenate_subgrids(results_subgrids, n_division)

In [ ]:
# results[1][1][1][1] == concatenate_subgrids(results_subgrids, n_division)[1][1][1][1]

In [ ]:
# results[1][1][1][1], concatenate_subgrids(results_subgrids, n_division)[1][1][1][1]

In [ ]:
conc = concatenate_subgrids(results_subgrids, n_division)
for o in range(results.shape[0]):
    for s in range(results.shape[0]):
            for q in range(results.shape[0]):
                for k in range(results.shape[0]):
                    for j in range(results.shape[0]):
                        for i in range(results.shape[0]):
                            if results[i,j,k,q,s,o] != conc[i,j,k,q,s,o]:
                                print(i,j,k,q,s,o)

In [ ]:
results[0,5,3,5,0,0]

In [ ]:
integral_calculation2(a0_lin[0], a1_lin[5], a2_lin[3], a3_lin[5], m_lin[0], z_lin[0])

----
----
----
----

In [ ]:
r = concatenate_subgrids(results_subgrids, n_division)

In [ ]:
r[9] == results[9]

In [ ]:
strips, bases = concatenate_subgrids(results_subgrids, n_division)

In [ ]:
for strip in strips:
    print(strip.shape)

In [ ]:
for base in bases:
    print(base.shape)

In [ ]:
np.concatenate(bases, axis=2).shape

In [ ]:
# def concatenate_subgrids(subgrids, nd):
#     n_subgrids = len(subgrids)
#     strips_array = []
#     bases_array = []
    

#     for i in range(0, n_subgrids-1, 2):
#         strips_array.append(np.concatenate((subgrids[i], subgrids[i+1]), axis=0))
        
#     for j in range(0, len(strips_array)-1, 2):
#         bases_array.append(np.concatenate((strips_array[j], strips_array[j+1]), axis=1))

        
#     return np.concatenate((bases_array[0], bases_array[1]), axis=2)

In [ ]:
concatenate_subgrids(results_subgrids,3)

In [ ]:
concatenate_subgrids(results_subgrids,3) == results

In [ ]:
# def concatenate_subgrids(subgrids, nd):
#     n_subgrids = len(subgrids)
#     strips_array = []
#     bases_array = []
    

#     for i in range(0, n_subgrids-1, 2):
#         strips_array.append(np.concatenate((subgrids[i][0], subgrids[i+1][0]), axis=0))
        
#     for j in range(0, len(strips_array)-1, 2):
#         bases_array.append(np.concatenate((strips_array[j], strips_array[j+1]), axis=1))

        
#     return np.concatenate((bases_array[0], bases_array[1]), axis=2)







#     strip1_x = np.concatenate((subgrids[0][0], subgrids[1][0]), axis=0)
#     strip2_x = np.concatenate((subgrids[2][0], subgrids[3][0]), axis=0)
#     strip3_x = np.concatenate((subgrids[4][0], subgrids[5][0]), axis=0)
#     strip4_x = np.concatenate((subgrids[6][0], subgrids[7][0]), axis=0)

#     base1_x = np.concatenate((strip1_x, strip2_x), axis=1)
#     base2_x = np.concatenate((strip3_x, strip4_x), axis=1)

#     full_x = np.concatenate((base1_x, base2_x), axis=2)

In [ ]:
concatenate_subgrids(results_subgrids,3)

## Re-concatenate grid_X

In [ ]:
strip1_x = np.concatenate((subgrids[0][0], subgrids[1][0]), axis=0)
strip2_x = np.concatenate((subgrids[2][0], subgrids[3][0]), axis=0)
strip3_x = np.concatenate((subgrids[4][0], subgrids[5][0]), axis=0)
strip4_x = np.concatenate((subgrids[6][0], subgrids[7][0]), axis=0)

base1_x = np.concatenate((strip1_x, strip2_x), axis=1)
base2_x = np.concatenate((strip3_x, strip4_x), axis=1)

full_x = np.concatenate((base1_x, base2_x), axis=2)

## Re-Concatenate grid_Y

In [ ]:
strip1_y = np.concatenate((subgrids[0][1], subgrids[1][1]), axis=0)
strip2_y = np.concatenate((subgrids[2][1], subgrids[3][1]), axis=0)
strip3_y = np.concatenate((subgrids[4][1], subgrids[5][1]), axis=0)
strip4_y = np.concatenate((subgrids[6][1], subgrids[7][1]), axis=0)

base1_y = np.concatenate((strip1_y, strip2_y), axis=1)
base2_y = np.concatenate((strip3_y, strip4_y), axis=1)

full_y = np.concatenate((base1_y, base2_y), axis=2)

## Re-Concatenate grid_Z

In [ ]:
strip1_z = np.concatenate((subgrids[0][2], subgrids[1][2]), axis=0)
strip2_z = np.concatenate((subgrids[2][2], subgrids[3][2]), axis=0)
strip3_z = np.concatenate((subgrids[4][2], subgrids[5][2]), axis=0)
strip4_z = np.concatenate((subgrids[6][2], subgrids[7][2]), axis=0)

base1_z = np.concatenate((strip1_z, strip2_z), axis=1)
base2_z = np.concatenate((strip3_z, strip4_z), axis=1)

full_z = np.concatenate((base1_z, base2_z), axis=2)

In [ ]:
concatenate_subgrids(subgrids, 3)

In [ ]:
strips[0]

In [ ]:
bases = []
for j in range(0, len(strips)-1, 2):
    # print(strips[j], strips[j+1])
    bases.append(np.concatenate((strips[j], strips[j+1]), axis=1))
    # print(j, j+1)

In [ ]:
np.concatenate((bases[0], bases[0]), axis=2)

In [ ]:
for i in range(0, 7-1, 2):
    print(i)

In [ ]:
strips_array = np.ndarray([], dtype=object)

In [ ]:
np.append(strips_array, [strip1_x])

In [ ]:
full_x.shape == grid_x.shape, full_y.shape == grid_y.shape, full_z.shape == grid_z.shape

In [ ]:
full_x == grid_x, full_y == grid_y, full_z == grid_z

In [ ]:
grid_y.shape == full_y.shape

In [ ]:
grid_y == full_y

-----
-----

In [ ]:
strip1 = np.concatenate((subgrids[0][0], subgrids[1]), axis=0)

strip2 = np.concatenate((subgrids[2], subgrids[3]), axis=0)

strip3 = np.concatenate((subgrids[4], subgrids[5]), axis=0)

strip4 = np.concatenate((subgrids[6], subgrids[7]), axis=0)

base1 = np.concatenate((strip1, strip2), axis=2)

base2 = np.concatenate((strip3, strip4), axis=2)

full = np.concatenate((base1, base2), axis=3)

In [ ]:
# for n, subgrid in enumerate(subgrids):
#     print(f"Subgrid {n+1}")
#     for k in range(2):
#         for j in range(2):
#             for i in range(2):
                
#                 print(subgrid[0][i,j,k], subgrid[1][i,j,k], subgrid[2][i,j,k])
#                 print()
                

In [ ]:
grid_x_concat = np.concatenate([subgrid[0] for subgrid in subgrids], axis=0)
grid_y_concat = np.concatenate([subgrid[1] for subgrid in subgrids], axis=0)
grid_z_concat = np.concatenate([subgrid[2] for subgrid in subgrids], axis=0)

# Print the concatenated grid shapes
print("Concatenated grid x shape:", grid_x_concat.shape)
print("Concatenated grid y shape:", grid_y_concat.shape)
print("Concatenated grid z shape:", grid_z_concat.shape)

In [ ]:
grid_x_concat

In [ ]:
subgrids[0]

In [ ]:

for n, subgrid in enumerate(subgrids):
    print(f"Subgrid {n+1}")
    for k in range(2):
        for j in range(2):
            for i in range(2):
                
                print(subgrid[0][i,j,k], subgrid[1][i,j,k], subgrid[2][i,j,k])
                print()
                

In [ ]:
concatenated_grid_x = np.concatenate([subgrid[0] for subgrid in subgrids], axis=0)
concatenated_grid_y = np.concatenate([subgrid[1] for subgrid in subgrids], axis=0)
concatenated_grid_z = np.concatenate([subgrid[2] for subgrid in subgrids], axis=0)

In [ ]:
concatenated_grid_x

In [ ]:
grid_x

In [ ]:
strip1.shape, strip2.shape, strip3.shape, strip4.shape, base1.shape, base2.shape, full.shape

In [ ]:
subgrids[0][0]

In [ ]:
for k in range(4):
        for j in range(4):
            for i in range(4):
                
                print(full[i,j,k], full[i,j,k], full[i,j,k])

In [ ]:
full

In [ ]:
grid_x, grid_y, grid_z

In [ ]:
subgrids[0]

In [ ]:
subgrids[0]

In [ ]:
grid_y

In [ ]:
grid_z

In [ ]:
subresults = []

for subgrid in subgrids:
    subresult = np.zeros((2,2,2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
#                 print('(',subgrid[0][i,j,k] + subgrid[1][i,j,k] + subgrid[2][i,j,k],')')
                subresult[i,j,k] = subgrid[0][i,j,k] + subgrid[1][i,j,k] + subgrid[2][i,j,k]
    subresults.append(subresult)

In [ ]:
subresults

In [ ]:
for subgrid in subgrids:
    for i in range(2):
        for j in range(2):
            for k in range(2):
                print('(',subgrid[0][i,j,k] + subgrid[1][i,j,k] + subgrid[2][i,j,k],')')

In [ ]:
# Define the submatrix size
submatrix_size = 4

# Divide the matrix into four adjacent 5x5 matrices
submatrices = []
for i in range(2):
    for j in range(2):
        submatrix = matrix[i*submatrix_size : (i+1)*submatrix_size, j*submatrix_size : (j+1)*submatrix_size]
        submatrices.append(submatrix)

# Print the submatrices
print(matrix)
for i, submatrix in enumerate(submatrices):
    print(f"Submatrix {i+1}:")
    print(submatrix)
    print()

In [ ]:
x = np.linspace(1,16,16)
y = np.linspace(1,16,16)


xx0, yy0 = np.meshgrid(x[0:4],y[0:4], indexing='ij')
xx1, yy1 = np.meshgrid(x[4:8],y[4:8], indexing='ij')
xx2, yy2 = np.meshgrid(x[8:12],y[8:12], indexing='ij')
xx3, yy2 = np.meshgrid(x[12:16],y[12:16], indexing='ij')

In [ ]:
results0 = np.zeros((xx0.shape[0], yy0.shape[0]))
for i in range(xx0.shape[0]):
    for j in range(yy0.shape[0]):
        results0[i,j] = xx0[i,j] * yy0[i,j]

In [ ]:
results0

In [ ]:
results1 = np.zeros((xx0.shape[0], yy0.shape[0]))
for i in range(xx0.shape[0]):
    for j in range(yy0.shape[0]):
        results1[i,j] = xx0[i,j] * yy1[i,j]

In [ ]:
results1

In [ ]:
np.concatenate((results0, results1), axis=1)

In [ ]:
results2 = np.zeros((xx0.shape[0], yy0.shape[0]))
for i in range(xx0.shape[0]):
    for j in range(yy0.shape[0]):
        results2[i,j] = xx1[i,j] * yy0[i,j]

In [ ]:
results2

In [ ]:
import numpy as np

# Create a 10x10 matrix
matrix = np.arange(1000).reshape(10, 10, 10)

print(matrix)
# # Define the submatrix size
# submatrix_size = 2

# # Divide the matrix into four adjacent 5x5 matrices
# submatrices = []
# for i in range(5):
#     for j in range(5):
#         submatrix = matrix[i*submatrix_size : (i+1)*submatrix_size, j*submatrix_size : (j+1)*submatrix_size]
#         submatrices.append(submatrix)

# # Print the submatrices
# print(matrix)
# for i, submatrix in enumerate(submatrices):
#     print(f"Submatrix {i+1}:")
#     print(submatrix)
#     print()


In [ ]:
import numpy as np

# Create a 10x10 matrix
matrix = np.arange(100).reshape(10, 10)

# Define the submatrix size
submatrix_size = 2

# Divide the matrix into four adjacent 5x5 matrices
submatrices = []
for i in range(5):
    for j in range(5):
        submatrix = matrix[i*submatrix_size : (i+1)*submatrix_size, j*submatrix_size : (j+1)*submatrix_size]
        submatrices.append(submatrix)

# Print the submatrices
print(matrix)
for i, submatrix in enumerate(submatrices):
    print(f"Submatrix {i+1}:")
    print(submatrix)
    print()


In [ ]:
c = np.concatenate((results0, results1), axis=1)
np.concatenate((c, results2), axis=0)

In [ ]:
for i in range(xx.shape[0]):
    for j in range(yy.shape[0]):
        for k in range(zz.shape[0]):
            print(xx[i,j,k], yy[i,j,k], zz[i,j,k])

In [ ]:
# Define cosmology
cosmo = FlatLambdaCDM(H0=67.8, Om0=0.307)

# Read hdf5 for BGS data
bgs = aTable.Table.read('BGS_ANY_full.provabgs.lite.hdf5')
is_bgs_bright = bgs['is_bgs_bright']
is_bgs_faint = bgs['is_bgs_faint']

bgs = bgs[bgs['is_bgs_bright']]

mask_zlim = (bgs['Z_HP'].data > 0.01)

z_tot = bgs['Z_HP'].data[mask_zlim]
x_tot = bgs['provabgs_logMstar'].data[mask_zlim]
x_median_tot = np.median(x_tot, axis=1)
w_zfail_tot = bgs['provabgs_w_zfail'].data[mask_zlim]
w_fib_tot = bgs['provabgs_w_fibassign'].data[mask_zlim]
vmax_tot = bgs['Vmax'].data[mask_zlim]

mask_mlim = []
for i in range(len(x_median_tot)):
    mask_mlim.append(x_median_tot[i] > mass_completeness_limit(z_tot[i]))
    

mask = (w_zfail_tot > 0) & (mask_mlim) & (z_tot<0.4)

z = z_tot[mask].astype(np.float32)
x = x_tot[mask].astype(np.float32)
x_median = x_median_tot[mask].astype(np.float32)
w_zfail = w_zfail_tot[mask].astype(np.float32)
w_fib = w_fib_tot[mask].astype(np.float32)
vmax = vmax_tot[mask].astype(np.float32)

# Spectroscopic weights
w_spec = (w_zfail*w_fib)

# integral_limits_list = []
# for z_i, m_i in zip(z, x_median):
#     integral_limits_list.append(integral_limits(z_i, m_i))

In [ ]:
nn = 4

a0_lin = np.linspace(9.5, 13., nn)
a1_lin = np.linspace(2.5, 6., nn)
a2_lin = np.linspace(-2.5, -0.5, nn)
a3_lin = np.linspace(0, 3., nn)
m_lin = np.linspace(x_median.min(), x_median.max(), nn)
z_lin = np.linspace(z.min(), z.max(), nn)

In [ ]:
a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin[0:2], a1_lin[0:2], 
                                               a2_lin[0:2], a3_lin[0:2], m_lin[0:2], z_lin[0:2],
                                               indexing='ij')

box1 = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                a0_g.shape[0], a0_g.shape[1]))

for i in range(a0_g.shape[0]):
    for j in range(a1_g.shape[1]):
        for k in range(a2_g.shape[2]):
            for l in range(a3_g.shape[3]):
                for q in range(m_g.shape[0]):
                    for t in range(z_g.shape[0]):

                        int_lim_gt = integral_limits(z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])

                        result = integral_calculation(int_lim_gt, a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                      a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t])

                        box1[i, j, k, l, q, t] = result


In [ ]:
# a0_lin,a0_lin[0:2]

In [ ]:
# for i in range(a0_lin.shape[0]-1):
#     print(a0_lin[i:i+2])

In [ ]:
a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin[2:4], a1_lin[2:4], 
                                               a2_lin[2:4], a3_lin[2:4], m_lin[2:4], z_lin[2:4],
                                               indexing='ij')

box2 = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                a0_g.shape[0], a0_g.shape[1]))

for i in range(a0_g.shape[0]):
    for j in range(a1_g.shape[1]):
        for k in range(a2_g.shape[2]):
            for l in range(a3_g.shape[3]):
                for q in range(m_g.shape[0]):
                    for t in range(z_g.shape[0]):

                        int_lim_gt = integral_limits(z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])

                        result = integral_calculation(int_lim_gt, a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                      a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t])

                        box2[i, j, k, l, q, t] = result


In [ ]:
a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin, a1_lin, 
                                               a2_lin, a3_lin, m_lin, z_lin,
                                               indexing='ij')

results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                a0_g.shape[0], a0_g.shape[1]))


for i in range(a0_g.shape[0]):
    for j in range(a1_g.shape[1]):
        for k in range(a2_g.shape[2]):
            for l in range(a3_g.shape[3]):
                for q in range(m_g.shape[0]):
                    for t in range(z_g.shape[0]):

                        int_lim_gt = integral_limits(z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])

                        result = integral_calculation(int_lim_gt, a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                      a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t])

                        results[i, j, k, l, q, t] = result

In [ ]:
box1

In [ ]:
box2

In [ ]:
np.concatenate((box1, box2), axis=0)

In [ ]:
box_tot = np.concatenate((box1, box2), axis=2)
box_tot = np.concatenate((box_tot, box_tot), axis=1)
box_tot = np.concatenate((box_tot, box_tot), axis=4)
box_tot = np.concatenate((box_tot, box_tot), axis=3)
box_tot = np.concatenate((box_tot, box_tot), axis=5)
box_tot = np.concatenate((box_tot, box_tot), axis=0)

In [ ]:
box_tot.shape

In [ ]:
box_tot == results

In [ ]:
box_tot

In [ ]:
results

In [ ]:
for b in range(int(nn/2)):
    a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin[b::2], a1_lin[b::2], 
                                                   a2_lin[b::2], a3_lin[b::2], m_lin[b::2], z_lin[b::2],
                                                   indexing='ij')
    
    results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                    a0_g.shape[0], a0_g.shape[1]))
    
    boxes = [np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                    a0_g.shape[0], a0_g.shape[1])), np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                    a0_g.shape[0], a0_g.shape[1]))]
    
    for i in range(a0_g.shape[0]):
        for j in range(a1_g.shape[1]):
            for k in range(a2_g.shape[2]):
                for l in range(a3_g.shape[3]):
                    for q in range(m_g.shape[0]):
                        for t in range(z_g.shape[0]):

                            int_lim_gt = integral_limits(z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])

                            result = integral_calculation(int_lim_gt, a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                          a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t])

                            results[i, j, k, l, q, t] = result
        
        boxes[i] = results

In [ ]:
boxes[1] == boxes[0]

In [ ]:
a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, m_lin, z_lin, indexing='ij')

results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                    a0_g.shape[0], a0_g.shape[1]))

for i in range(a0_g.shape[0]):
    for j in range(a1_g.shape[1]):
        for k in range(a2_g.shape[2]):
            for l in range(a3_g.shape[3]):
                for q in range(m_g.shape[0]):
                    for t in range(z_g.shape[0]):

                        int_lim_gt = integral_limits(z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])

                        result = integral_calculation(int_lim_gt, a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                      a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t])

                        results[i, j, k, l, q, t] = result

In [ ]:
box_tot = np.concatenate((boxes[0], boxes[1]), dtype=object, axis=0)

In [ ]:
box_tot.shape

In [ ]:
boxes[0] == boxes[1]

In [ ]:
box_tot == results

In [ ]:
box_tot.shape

In [ ]:
results.shape

In [ ]:
a0_lin

In [ ]:
a0_lin[0::2], a0_lin[1::2]

In [ ]:
a1_lin = np.linspace(-4., 0., nn)
m_lin = np.linspace(x_median.min(), (x_median.max() - x_median.min())/2., nn)
z_lin = np.linspace(z.min(), (z.max() - z.min())/2., nn)

In [ ]:
xx, yy, mm, zz = np.meshgrid(a0_lin, a1_lin, m_lin, z_lin, indexing='ij')
box1 = np.zeros((xx.shape[0], yy.shape[0], mm.shape[0], zz.shape[0]))

In [ ]:
xx

In [ ]:
a0_lin2 = np.linspace(17.5, 21.5, nn)
a1_lin2 = np.linspace(12, 20, nn)


xx2, yy2, x = np.meshgrid(a0_lin2, a1_lin2, m_lin, z_lin, indexing='ij')
box2 = np.zeros((xx2.shape[0], yy2.shape[0]))

In [ ]:
for i in range(xx.shape[0]):
    for j in range(yy.shape[0]):
        print(xx[i,j], yy[i,j])

In [ ]:
xx, xx2

In [ ]:
np.concatenate((xx,xx2), dtype=object)

In [ ]:
nn = 11

a0_lin = np.linspace(9.5, 13.5, nn)
a1_lin = np.linspace(-4., 4., nn)
a2_lin = np.linspace(-2.5, -0.5, nn)
a3_lin = np.linspace(1.5, 6., nn)
m_lin = np.linspace(x_median.min(), x_median.max(), nn)
z_lin = np.linspace(z.min(), z.max(), nn)

# m_g, z_g = np.meshgrid(m_lin, z_lin, indexing='ij')
# integral_lim_matrix = np.zeros((m_g.shape[0], z_g.shape[0]), dtype=np.ndarray)

# for i in range(m_g.shape[0]):
#     for j in range(z_g.shape[0]):
#         integral_lim_matrix[i,j] = integral_limits(z_g[i,j], m_g[i,j])
        
# a0_g, a1_g, a2_g, a3_g, int_lim_g = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, integral_lim_matrix,
#                                                     indexing='ij')

In [ ]:
grid_data = np.load('grid_6d_11sample_emcee.npy')

In [ ]:
interp = RegularGridInterpolator((a0_lin, a1_lin, a2_lin, a3_lin, m_lin, z_lin), grid_data)

In [ ]:
grid_data.min(), grid_data.max()

In [ ]:
grid_data[0,:,:,:,:,:].min()

In [ ]:
grid_data[0,:,:,:,:,:]

In [ ]:
def log_likelihood(a0, a1, a2, a3, interp, x_median,  w, z, x):
    q = smf_single_schechter_sty(x, z, a0, a1, a2, a3)
    a0_v = np.repeat(a0, x_median.shape[0])
    a1_v = np.repeat(a1, x_median.shape[0])
    a2_v = np.repeat(a2, x_median.shape[0])
    a3_v = np.repeat(a3, x_median.shape[0])
    
    pt = np.array([a0_v, a1_v, a2_v, a3_v, x_median, z]).T
    I = interp(pt)
    
    a = np.log10(np.sum(q, axis=1)) - np.log10(I)
    return a * w

In [ ]:
log_likelihood(9,0.3,-1,3, interp, x_median, w_spec, z, x)

In [ ]:
interp(np.array([9.,0.3, -1, 3, 10., 0.2]))

In [ ]:
interp(np.array([13.,0.5, 0., 4., x_median.max(), z.max()]))

In [ ]:
a0_v = np.repeat(8., x_median.shape[0])
a1_v = np.repeat(0.3, x_median.shape[0])
a2_v = np.repeat(-1, x_median.shape[0])
a3_v = np.repeat(3, x_median.shape[0])

In [ ]:
pt = np.array([a0_v, a1_v, a2_v, a3_v, x_median, z])

In [ ]:
pt.T

In [ ]:
interp(pt.T)

In [ ]:
pt = np.array([[10, 10],[0.3, 0.3],[-1, -1],[3,3],[10., 10],[0.2,0.2]])

In [ ]:
pt.T

In [ ]:
interp(pt.T)

In [ ]:
pt = np.array([10,0.3,-1,3,10.,0.2])
print(integral_calculation2(10,0.3,-1,3,0.2, 10))
print(interp(pt))
print(integral_calculation2(10,0.3,-1,3,0.2, 10) - interp(pt))

---
---
----

In [ ]:
start_time = time.time()
a0_lin = np.linspace(9, 13, 2)
a1_lin = np.linspace(0, 0.5, 2)
a2_lin = np.linspace(-3, 0, 2)
a3_lin = np.linspace(2, 4, 2)
m_lin = np.linspace(x_median.min(), x_median.max(), 2)
z_lin = np.linspace(z.min(), z.max(), 2)

a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, m_lin, z_lin, indexing='ij')

results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3],
                    a0_g.shape[0], a0_g.shape[1]))

for i in range(a0_g.shape[0]):
    for j in range(a1_g.shape[1]):
        for k in range(a2_g.shape[2]):
            for l in range(a3_g.shape[3]):
                for q in range(m_g.shape[0]):
                    for t in range(z_g.shape[0]):
#                     print(a0_g[i,j,k,l,q], a1_g[i,j,k,l,q], 
#                           a2_g[i,j,k,l,q], a3_g[i,j,k,l,q], int_lim_g[i,j,k,l,q])
#                     print(i,j,k,l,q)
                        int_lim_gt = integral_limits(z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])
                        print(int_lim_gt, '\n')
                        result = integral_calculation(int_lim_gt, a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                      a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t])
                        results[i, j, k, l, q, t] = result
            
end_time = time.time()
print(" took {0:.1f} min ".format((end_time-start_time)/60.))

In [ ]:
integral_lim_matrix

In [ ]:
for i in range(a0_g.shape[0]):
    for j in range(a1_g.shape[1]):
        for k in range(a2_g.shape[2]):
            for l in range(a3_g.shape[3]):
                for q in range(m_g.shape[0]):
                    for t in range(z_g.shape[0]):
                        print(results[i,j,k,l,q,t], ' - ', a0_g[i, j, k, l, q, t], a1_g[i, j, k, l, q, t],
                                                      a2_g[i, j, k, l, q, t], a3_g[i, j, k, l, q, t], 
                             z_g[i, j, k, l, q, t], m_g[i, j, k, l, q, t])

In [ ]:
results

In [ ]:
z_g[0,0]

In [ ]:
integral_calculation(integral_limits_list[0],a0_lin[0], a1_lin[0], a2_lin[0], a3_lin[0])

In [ ]:
# 6D GRID  (a0, a1, a2, a3, mstar, z)

n_sample_per_dim = [2,3]
times_list = []
for nn in n_sample_per_dim:
    start_time = time.time()
    a0_lin = np.linspace(9, 13, nn)
    a1_lin = np.linspace(0, 0.5, nn)
    a2_lin = np.linspace(-3, 0, nn)
    a3_lin = np.linspace(2, 4, nn)
    m_lin = np.linspace(x_median.min(), x_median.max(), nn)
    z_lin = np.linspace(z.min(), z.max(), nn)

    # a0_g, a1_g, a2_g, a3_g, m_g, z_g = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, m_lin, z_lin, indexing='ij')

    m_g, z_g = np.meshgrid(m_lin, z_lin, indexing='ij')
    integral_lim_matrix = np.zeros((m_g.shape[0], z_g.shape[0]), dtype=np.ndarray)

    for i in range(m_g.shape[0]):
        for j in range(z_g.shape[0]):
            integral_lim_matrix[i,j] = integral_limits(z_g[i,j], m_g[i,j])

    a0_g, a1_g, a2_g, a3_g, int_lim_g = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, integral_lim_matrix,
                                                    indexing='ij')

    
    start_time = time.time()

    # Create an array to store the results
    results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3], int_lim_g.shape[0]))

    # Evaluate the function on each point of the grid

    for i in range(a0_g.shape[0]):
        for j in range(a1_g.shape[1]):
            for k in range(a2_g.shape[2]):
                for l in range(a3_g.shape[3]):
                    for q in range(int_lim_g.shape[0]):

    #                     print(a0_g[i,j,k,l,q], a1_g[i,j,k,l,q], 
    #                           a2_g[i,j,k,l,q], a3_g[i,j,k,l,q], int_lim_g[i,j,k,l,q])
                        print(i,j,k,l,q)
                        result = integral_calculation(int_lim_g[i,j,k,l,q], a0_g[i, j, k, l,q], a1_g[i, j, k, l,q],
                                                          a2_g[i, j, k, l,q], a3_g[i, j, k, l,q])


                        results[i, j, k, l, q] = result


    end_time = time.time()
    times_list.append((end_time-start_time)/60)

In [ ]:
int_lim_g[0,0,0,0,0], a0_g[0,0,0,0,0], a1_g[0,0,0,0,0], a2_g[0,0,0,0,0], a3_g[0,0,0,0,0]

In [ ]:
integral_limits_list[0],a0_lin[0], a1_lin[0], a2_lin[0], a3_lin[0]

In [ ]:
integral_calculation(int_lim_g[0,0,0,0,0], a0_g[0,0,0,0,0], a1_g[0,0,0,0,0], a2_g[0,0,0,0,0], a3_g[0,0,0,0,0])

In [ ]:
print(times_list)

In [ ]:
print(" took {0:.1f} min".format((end_time-start_time)/60))

In [ ]:
results

In [ ]:
results[0,0,0,0,0]

In [ ]:
# 4D GRID (a0, a1, a2, a3) JUST ONE GALAXY

start_time = time.time()
a0_lin = np.linspace(9, 13, 20)
a1_lin = np.linspace(0, 0.5, 20)
a2_lin = np.linspace(-3, 0, 20)
a3_lin = np.linspace(2, 4, 20)


a0_g, a1_g, a2_g, a3_g = np.meshgrid(a0_lin, a1_lin, a2_lin, a3_lin, indexing='ij')


# Create an array to store the results
results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3]))

# Evaluate the function on each point of the grid
for i in range(a0_g.shape[0]):
    for j in range(a0_g.shape[1]):
        for k in range(a0_g.shape[2]):
            for l in range(a0_g.shape[3]):
                
                
                result = integral_calculation(integral_limits_list[0], 
                                              a0_g[i, j, k, l], a1_g[i, j, k, l],
                                              a2_g[i, j, k, l], a3_g[i, j, k, l])

                results[i, j, k, l] = result

# Print the results
# print(results)
end_time = time.time()

In [ ]:
print(" took {0:.1f} min".format((end_time-start_time)/60))

In [ ]:
def integral_calculation_pool(params):
    limits, a0, a1, a2, a3 = params
    result = 0
    result += integrate.dblquad(smf_single_schechter_integral, limits[0][0],
                                limits[0][1], limits[1][0], limits[1][1], args=(a0, a1, a2, a3))[0]
    result += integrate.dblquad(smf_single_schechter_integral, limits[2][0],
                            limits[2][1], limits[3][0] , limits[3][1], args=(a0, a1, a2, a3))[0]
    
    return result

def evaluate_integral_pool(params):
    return integral_calculation_pool(params)

In [ ]:
para = [integral_limits_list[0], a0_g[i, j, k, l], a1_g[i, j, k, l], a2_g[i, j, k, l], a3_g[i, j, k, l]]

num_processes = 4
pool = Pool(processes=num_processes)

results = np.zeros((a0_g.shape[0], a0_g.shape[1], a0_g.shape[2], a0_g.shape[3]))
start_time = time.time()
# Evaluate the function on each point of the grid
for i in range(a0_g.shape[0]):
    for j in range(a0_g.shape[1]):
        for k in range(a0_g.shape[2]):
            for l in range(a0_g.shape[3]):
                
                
                result = pool.map(evaluate_integral_pool,para)

                results[i, j, k, l] = result

# Print the results
# print(results)
end_time = time.time()
print("With parallelisation took {0:.1f} min".format((end_time-start_time)/60))